In [4]:
import pandas as pd
from transformers import pipeline

# Load the dataset
file_path = "/kaggle/input/data-chatgpt/data_chatgpt.csv"  # Update path if needed
df = pd.read_csv(file_path, encoding='ISO-8859-1')  # Fixes UnicodeDecodeError

# Load the empathy classifier model
classifier = pipeline("text-classification", model="mrm8488/t5-base-finetuned-emotion", return_all_scores=True)

# Function to get empathy score (using 'sadness', 'fear', etc.)
def get_empathy_score(text):
    try:
        result = classifier(text[:512])[0]  # Truncate to avoid max length error
        empathy_emotions = ["sadness", "fear", "joy", "love"]  # Emotions we may care about
        scores = {item['label']: item['score'] for item in result if item['label'] in empathy_emotions}
        # Return highest scoring empathy-related emotion and its score
        if scores:
            top_emotion = max(scores, key=scores.get)
            return pd.Series([top_emotion, scores[top_emotion]])
        else:
            return pd.Series(["none", 0.0])
    except:
        return pd.Series(["error", 0.0])

# Apply to both doctor (output) and LLaMA responses
df[['doctor_emotion', 'doctor_empathy_score']] = df['output'].apply(get_empathy_score)
df[['llama_emotion', 'llama_empathy_score']] = df['llama response '].apply(get_empathy_score)

# Save result
df.to_csv("/kaggle/working/empathy_comparison_results.csv", index=False)

print("✅ Empathy analysis complete. Results saved as empathy_comparison_results.csv")


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at mrm8488/t5-base-finetuned-emotion and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Device set to use cpu
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


✅ Empathy analysis complete. Results saved as empathy_comparison_results.csv


In [8]:
import pandas as pd
from transformers import pipeline

# Load the dataset
file_path = "/kaggle/input/data-chatgpt/data_chatgpt.csv"  # Use the correct path
df = pd.read_csv(file_path, encoding='ISO-8859-1')  # Use correct encoding to avoid decode errors

# Load a better emotion detection model
classifier = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion")

# Function to predict empathy (label and score) with input truncation
def is_empathetic(text):
    text = str(text)[:300]  # Truncate long medical responses to 300 characters
    result = classifier(text)[0]
    return result['label'], result['score']

# Apply to both doctor and ChatGPT responses
df["doctor_empathy_label"], df["doctor_empathy_score"] = zip(*df["output"].apply(is_empathetic))
df["chatgpt_empathy_label"], df["chatgpt_empathy_score"] = zip(*df["llama response "].apply(is_empathetic))

# Save results
df.to_csv("/kaggle/working/empathy_comparison_results.csv", index=False)
print("✅ Empathy comparison complete and saved!")


Device set to use cpu


✅ Empathy comparison complete and saved!


In [10]:
import pandas as pd
from transformers import pipeline

# Load the dataset
file_path = "/kaggle/input/data-chatgpt/data_chatgpt.csv"  # Use the correct path
df = pd.read_csv(file_path, encoding='ISO-8859-1')  # Handle encoding errors

# Load the emotion detection model
classifier = pipeline("text-classification", model="nateraw/bert-base-uncased-emotion")

# Function to predict empathy (label and score) with input truncation
def is_empathetic(text):
    text = str(text)[:300]  # Truncate long medical responses
    result = classifier(text)[0]
    return result['label'], result['score']

# Apply model to both doctor and ChatGPT responses
df["doctor_empathy_label"], df["doctor_empathy_score"] = zip(*df["output"].apply(is_empathetic))
df["chatgpt_empathy_label"], df["chatgpt_empathy_score"] = zip(*df["llama response "].apply(is_empathetic))

# Save results
df.to_csv("/kaggle/working/empathy_comparison_results.csv", index=False)
print("✅ Empathy comparison complete and saved!")


Device set to use cpu


✅ Empathy comparison complete and saved!


We tested three emotion detection models: nateraw/bert-base-uncased-emotion, bhadresh-savani/distilbert-base-uncased-emotion, and mrm8488/t5-base-finetuned-emotion. All models classify emotions such as joy, sadness, anger, and fear, which we used to approximate empathy in patient-doctor and ChatGPT responses. The T5 model, while promising, generated inconsistent outputs and often returned incomplete or irrelevant labels, making it unsuitable for our evaluation. The nateraw model also underperformed by frequently labeling sensitive medical queries as "joy," which didn't align with the context. Ultimately, the bhadresh-savani model produced more realistic and relevant labels—like sadness and fear—making it the most reliable empathy classifier for our project.
